# Random Forest CSEs

This notebook is an exploration of using a Random Forest classifier and regressor in applying individual CSE decisions.  In this approach/test we are *only* concerned about a single individual choice from a method which is JIT'ed with no CSEs to a method that is JIT'ed with a single CSE.

Our goal in this exploration is to set a baseline for how good random forests can be at selecting a single CSE to enable or disable.  If we cannot use a random forest to classify whether we should or should not enable a single CSE, then we likely have either chosen the wrong data to try to make the decision, or this problem may not be a good fit for machine learning.

This exploration isn't meant to be the solution we immediately implement in the JIT.  This isn't intended to pick a sequence of CSEs to JIT.  Figuring out how we would use this (or a neural network) to make a series of CSE decisions will come later.

### Approaches

There are two approaches we are exploring here:

**classification** - We will attempt to classify CSEs into ones we should apply and CSEs we should not apply.  To do that, we've picked a `CSE_SUCCESS_THRESHOLD` semi-arbitrarily.  Any CSE that is applied that lowers the perf_score by at least that amount is considered a "successful" CSE choice.  Our goal with classification will be to use a random forest to predict whether we should

**regression** - We will also attempt to apply a random forest regressor to try to estimate the change in perf_score.  It will be much harder for any model to estimate the change in perf score than it will be to simply guess directionality.

### Implementation

To explore this space, we JIT every method in a `.mch` file, first with no CSEs enabled to get a baseline score.  Then we individually enable each viable CSE.  This takes several hours to finish, but the resulting data lets us know the decisions for each *individual* CSE choice, if they were selected first.  We then train on the individual selection of CSEs here.

### Limitations

This approach means that we do not have any CSE data with other CSEs already selected.  This is something we can attempt to address in the future.  Again, this isn't meant as a "let's go put this in the JIT today", only the next step in a long line of things to try and see what works.

Random forest classifiers are not explainable.  Like neural networks, they do not lend themselves *directly* to figuring out why a particular decision was made.

### Longer Term

While not a part of this exploration/checkin, we can use the results here in several ways:

First, even though random forests are not directly explainable, we can attempt to use libraries like `lime` to attempt to figure out why our classification random forest correctly identified 'bad' decisions which the current JIT heuristic decided to emit.  When our classifier gets it right but the heuristic gets it wrong, we can use the model to attempt to determine what criteria the heuristic *should* have used to figure out it was a bad choice.

This lets us improve our current heuristic using these results, even if we end up not using direct ML models in the JIT.

Second, one possible use of the random forest regressor is to reject catastrophic decisions in the current heuristic.  For example, after the JIT heuristic selects a CSE, run it through the random forest regressor and see if it thinks this decision will result in large increase in perfscore, and we could then reject the heuristic's choice.

This would put the heuristic still in the driver's seat, with an ML sanity check at the end.

Third, if we do want to directly use these models, they can be converted to ONNX and loaded directly into the JIT via Microsoft's ONNX library.  No need for python or tensorflow.

Fourth, and finally, this gives us a baseline to compare a neural network based approach to.  This helps us define what a reasonable result is as we experiment with neural network architectures.

In [1]:
# --------------------------------------------------------------------------------------------
# Constants and parameters

# Update these values to point at your local build
MCH_FILE = "~/git/runtime/artifacts/spmi/mch/43854594-cd60-45df-a89f-5b7697586f46.linux.x64/libraries_tests_no_tiered_compilation.run.linux.x64.Release.mch"
CORE_ROOT = "~/git/runtime/artifacts/bin/coreclr/linux.x64.Checked/"

# At what perf_score would we want to select a CSE?  We don't want to select any CSE which
# is 0.0, as we are creating new temporaries for no value.
# I've arbitrarily selected this minimum perf_score improvement for a "successful" CSE.
CSE_SUCCESS_THRESHOLD = -5.0

# We will only retain features which have a correlation of at least 1% with the change in
# perf_score.  This is to reduce the number of features we have to consider.  Selecting a
# value as high as 15% would still be reasonable, but since there are so few features we
# can afford to allow a lower threshold.
CORRELATION_THRESHOLD = 0.01

# We will use a 80/20 train/test split.
TRAIN_TEST_SPLIT = 0.2


# --------------------------------------------------------------------------------------------
# Setup and imports

# resolve '~' to home directory
import os
MCH_FILE = os.path.expanduser(MCH_FILE)
CORE_ROOT = os.path.expanduser(CORE_ROOT)

import os
import sys
from pandas import DataFrame, get_dummies

# Add the parent directory to the path so that we can import the jitml module
sys.path.append(os.path.dirname(os.getcwd()))
print(f"added: {os.path.dirname(os.getcwd())} to sys.path")

from jitml import get_individual_cse_perf


added: /home/leculver/git/jitml to sys.path


2024-05-14 10:32:17.693538: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 10:32:17.725830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 10:32:17.725862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 10:32:17.726695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 10:32:17.732030: I tensorflow/core/platform/cpu_feature_guar

## Load Individual CSE Information

The `get_individual_cse_perf` method will go through and JIT every method with each possible viable CSE selected one at a time.  So if a method has CSEs 0, 1, and 3 that are viable, it will JIT the method with no CSEs, only CSE 0 selected, only CSE 1 selected, and only CSE 3 selected.  It does not attempt to combine any CSE choices together.

Note that this can take hours to run the first time.  We have to JIT hundreds of thousands of methods, and `superpmi` streaming does have overhead.  The first call to `get_individual_cse_perf` can take hours to complete (it will show a progress bar if it's JIT'ing).  Subsequent calls to this method will return a cached result (the cached result is saved in a .csv file next to the .mch file).

The `cse_df` variable is a `pandas.DataFrame` object.  We will be using pandas for of our data processing.

In [2]:
# Load or calculate CSE perf_score data.
cse_df : DataFrame = get_individual_cse_perf(MCH_FILE, CORE_ROOT)
cse_df

,method,cse_index,no_cse_score,heuristic_score,cse_score,heuristic_selected,index,applied,viable,live_across_call,...,cost_sz,use_count,def_count,use_wt_cnt,def_wt_cnt,distinct_locals,local_occurrences,bb_count,block_spread,enreg_count
0,5,0,58.12,47.88,54.12,False,0,False,True,False,...,18,1,1,100,100,0,0,17,0,8
1,5,1,58.12,47.88,57.88,True,1,False,True,False,...,10,1,1,100,100,0,0,17,0,8
2,5,2,58.12,47.88,55.88,False,2,False,True,False,...,12,1,1,100,100,0,0,17,0,8
3,5,3,58.12,47.88,51.88,False,3,False,True,False,...,22,1,1,100,100,0,0,17,0,8
4,5,4,58.12,47.88,54.12,False,4,False,True,True,...,18,2,1,100,100,0,0,17,13,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535401,306741,2,155.09,155.09,155.09,False,2,False,True,False,...,12,1,1,100,100,0,0,17,0,27
535402,306741,3,155.09,155.09,155.09,True,3,False,True,False,...,10,1,1,45,4,0,0,17,2,27
535403,306741,4,155.09,155.09,154.34,True,4,False,True,False,...,4,1,1,45,100,1,1,17,2,27
535404,306741,6,155.09,155.09,154.34,False,6,False,True,False,...,4,1,1,45,100,1,1,17,2,27


## Defining a Successful CSE

The DataFrame object has some extra data we don't need, and some columns that aren't in the right format.  This section creates the data we are comparing against, and strips out those unneeded columns.

First, we define the `change` column.  This is the value the regression model will attempt to predict.  Second, we will use `change` along with  `CSE_SUCCESS_THRESHOLD` to build the `success` column, which is what the classification model will attempt to predict.  Then finally we will drop some columns that come from `get_individual_cse_perf` that we don't need or don't want to give the models.

In [3]:
# Build a new column 'success' which is True if the difference in perf_score is below the threshold

cse_df['change'] = cse_df['cse_score'] - cse_df['no_cse_score']
cse_df['success'] = cse_df['change'] < CSE_SUCCESS_THRESHOLD

print(f"Number of successful CSEs: {cse_df['success'].sum()}")
print(f"Number of unsuccessful CSEs: {len(cse_df) - cse_df['success'].sum()}")

# Filter out columns we don't need.
to_filter_out = ['method', 'cse_index', 'cse_score', 'no_cse_score', 'heuristic_score', 'heuristic_selected', 'index',
                 'applied', 'viable']

cse_df.drop(columns=to_filter_out, inplace=True)
cse_df

Number of successful CSEs: 55104
Number of unsuccessful CSEs: 480302


,live_across_call,const,shared_const,make_cse,has_call,containable,type,cost_ex,cost_sz,use_count,def_count,use_wt_cnt,def_wt_cnt,distinct_locals,local_occurrences,bb_count,block_spread,enreg_count,change,success
0,False,False,False,False,True,False,JitType.LONG,17,18,1,1,100,100,0,0,17,0,8,-4.00,False
1,False,True,False,False,False,False,JitType.LONG,3,10,1,1,100,100,0,0,17,0,8,-0.24,False
2,False,False,False,False,False,False,JitType.OTHER,5,12,1,1,100,100,0,0,17,0,8,-2.24,False
3,False,False,False,False,False,False,JitType.OTHER,8,22,1,1,100,100,0,0,17,0,8,-6.24,True
4,True,False,False,False,True,False,JitType.LONG,17,18,2,1,100,100,0,0,17,13,8,-4.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535401,False,False,False,False,False,False,JitType.OTHER,5,12,1,1,100,100,0,0,17,0,27,0.00,False
535402,False,True,False,False,False,False,JitType.OTHER,3,10,1,1,45,4,0,0,17,2,27,0.00,False
535403,False,False,False,False,False,False,JitType.INT,5,4,1,1,45,100,1,1,17,2,27,-0.75,False
535404,False,False,False,False,False,False,JitType.INT,5,4,1,1,45,100,1,1,17,2,27,-0.75,False


## Fixing 'type'

The CSE's expression `type` is currently an enum of values 0, 1, 2, etc.  This is not a scalar value where the increasing sequence is meaningful.  Instead we will one-hot encode this method with `pandas.get_dummies`, creating a True or False for each possible value.  This is much easier for a model to make sense of.

In [4]:
# Show the values of the 'type' column
print(cse_df['type'].value_counts())

# One-hot encode the 'type' column
cse_df = get_dummies(cse_df, columns=['type'])
cse_df

type
JitType.LONG      222219
JitType.OTHER     221903
JitType.INT        85602
JitType.DOUBLE      2501
JitType.FLOAT       2340
JitType.SIMD         841
Name: count, dtype: int64


,live_across_call,const,shared_const,make_cse,has_call,containable,cost_ex,cost_sz,use_count,def_count,...,block_spread,enreg_count,change,success,type_JitType.DOUBLE,type_JitType.FLOAT,type_JitType.INT,type_JitType.LONG,type_JitType.OTHER,type_JitType.SIMD
0,False,False,False,False,True,False,17,18,1,1,...,0,8,-4.00,False,False,False,False,True,False,False
1,False,True,False,False,False,False,3,10,1,1,...,0,8,-0.24,False,False,False,False,True,False,False
2,False,False,False,False,False,False,5,12,1,1,...,0,8,-2.24,False,False,False,False,False,True,False
3,False,False,False,False,False,False,8,22,1,1,...,0,8,-6.24,True,False,False,False,False,True,False
4,True,False,False,False,True,False,17,18,2,1,...,13,8,-4.00,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535401,False,False,False,False,False,False,5,12,1,1,...,0,27,0.00,False,False,False,False,False,True,False
535402,False,True,False,False,False,False,3,10,1,1,...,2,27,0.00,False,False,False,False,False,True,False
535403,False,False,False,False,False,False,5,4,1,1,...,2,27,-0.75,False,False,False,True,False,False,False
535404,False,False,False,False,False,False,5,4,1,1,...,2,27,-0.75,False,False,False,True,False,False,False


## Finding Correlated Features

We do not want to feed the model data that isn't correlated with the data we are looking for.

First, we will drop any columns with a single value.  In this case, 'shared_const' is always false, which is something to follow up on.

Second, we will find any column which does not have a strong correlation to `change` (positive or negative) above the threshold we defined in the constants above.  This will give us fewer features to train on and make the model's job easier.

In [5]:
# find columns with only a single value
to_drop = [col for col in cse_df.columns if len(cse_df[col].unique()) == 1]
print(f"Dropping columns with only one unique value: {to_drop}")
print()

cse_df.drop(columns=to_drop, inplace=True)

# This function in pandas.DataFrame returns a Series with the correlation of each column with
# every other column.  We'll filter that down to just correlation with the 'success' column.
# We also don't care about the direction of the correlation, only that it's correlated.
correlation = cse_df.corr()['success'].apply(abs)

print("Column correlation with 'success':")
print(correlation.sort_values(ascending=False))

# Filter out columns which have a correlation below the threshold
to_drop = correlation < CORRELATION_THRESHOLD
to_drop = to_drop[to_drop].index
if len(to_drop) > 0:
    print(f"Dropping columns with a correlation below the threshold:")
    for col in to_drop:
        print(f"  {correlation[col]:.4f} {col}")

cse_df.drop(columns=to_drop, inplace=True)

Dropping columns with only one unique value: ['shared_const']

Column correlation with 'success':
success                1.000000
const                  0.315422
local_occurrences      0.314266
distinct_locals        0.313431
cost_ex                0.227591
type_JitType.INT       0.225176
make_cse               0.186611
cost_sz                0.154731
has_call               0.144333
bb_count               0.125688
type_JitType.LONG      0.102867
live_across_call       0.098688
block_spread           0.070130
type_JitType.OTHER     0.064999
use_count              0.062435
change                 0.043882
def_wt_cnt             0.041071
use_wt_cnt             0.038193
enreg_count            0.031402
containable            0.013465
type_JitType.DOUBLE    0.010223
def_count              0.009769
type_JitType.SIMD      0.009382
type_JitType.FLOAT     0.007657
Name: success, dtype: float64
Dropping columns with a correlation below the threshold:
  0.0098 def_count
  0.0077 type_JitType.FLOAT


## Build a Random Forest Classifier

The scikit-learn package has excellent random forest implementations.  We will first use it to build a classification model from the features we have distilled down.  Note that we have two output labels we are trying to predict, one for each model, so we have to filter out `success` and `change` from both, while picking hte correct one per model.

The results here are quite good.  A random forest classifier is 98% accurate in guessing whether an individual CSE (with none chosen) will be above or below our `CSE_SUCCESS_THRESHOLD`.

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x_c, y_c = cse_df.drop(columns=['success', 'change'], inplace=False), cse_df['success']
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(x_c, y_c, test_size=TRAIN_TEST_SPLIT, random_state=42)

scaler = StandardScaler()
x_train_c = scaler.fit_transform(x_train_c)
x_test_c = scaler.transform(x_test_c)

classifier = RandomForestClassifier(n_jobs=-1)
classifier.fit(x_train_c, y_train_c)

print(f"Train score: {classifier.score(x_train_c, y_train_c):.4f}")
print(f"Test score: {classifier.score(x_test_c, y_test_c):.4f}")
print()
print("Feature importance:")
for col, importance in sorted(zip(x_c.columns, classifier.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"{importance:.4f} {col}")

Train score: 0.9988
Test score: 0.9833

Feature importance:
0.2929 use_wt_cnt
0.1120 def_wt_cnt
0.0822 cost_ex
0.0713 enreg_count
0.0689 bb_count
0.0609 cost_sz
0.0531 block_spread
0.0476 use_count
0.0390 local_occurrences
0.0388 const
0.0315 distinct_locals
0.0236 containable
0.0197 live_across_call
0.0185 type_JitType.INT
0.0114 type_JitType.LONG
0.0105 make_cse
0.0091 has_call
0.0084 type_JitType.OTHER
0.0005 type_JitType.DOUBLE


## Build a Random Forest Regressor

Building a regressor instead of a classifier simply requires using `RandomForestRegressor` to predict `change`:

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

x_r, y_r = cse_df.drop(columns=['success', 'change'], inplace=False), cse_df['change']
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(x_r, y_r, test_size=TRAIN_TEST_SPLIT, random_state=42)

scaler = StandardScaler()
x_train_r = scaler.fit_transform(x_train_r)
x_test_r = scaler.transform(x_test_r)

regressor = RandomForestRegressor(n_jobs=-1)
regressor.fit(x_train_r, y_train_r)

print(f"Train score: {regressor.score(x_train_r, y_train_r):.4f}")
print(f"Test score: {regressor.score(x_test_r, y_test_r):.4f}")
print()

print("Feature importance:")
for col, importance in sorted(zip(x_c.columns, regressor.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"{importance:.4f} {col}")

Train score: 0.9564
Test score: 0.5019

Feature importance:
0.8659 use_wt_cnt
0.0309 block_spread
0.0193 def_wt_cnt
0.0175 const
0.0086 cost_ex
0.0071 enreg_count
0.0070 use_count
0.0068 containable
0.0068 cost_sz
0.0063 type_JitType.OTHER
0.0062 bb_count
0.0052 type_JitType.INT
0.0041 make_cse
0.0030 local_occurrences
0.0021 type_JitType.LONG
0.0019 live_across_call
0.0013 distinct_locals
0.0001 has_call
0.0000 type_JitType.DOUBLE


Unfortunately, this model seems to overtrain without hyperparameter tuning.  The end result here is a 96% success rate on training data, but a 57% success rate on test data which is a classic sign of overtraining.

We can search for the best hyperparameters by using `GridSearchCV`.  However, this will take many hours to find the best parameters:

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the RandomForestRegressor
regressor = RandomForestRegressor(n_jobs=-1, random_state=42)

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=1, scoring='r2')

# Fit the model
grid_search.fit(x_train_r, y_train_r)

# Output the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Get the best estimator
best_regressor = grid_search.best_estimator_

# Evaluate the best model
print(f"Train score: {best_regressor.score(x_train_r, y_train_r):.4f}")
print(f"Test score: {best_regressor.score(x_test_r, y_test_r):.4f}")
print()

print("Feature importance:")
for col, importance in sorted(zip(x_r.columns, best_regressor.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"{importance:.4f} {col}")
```

## Conclusions

A random forest classifier can be built quickly and easily to determine if we should choose an individual CSE when none are selected.  This, of course, does not solve the general problem of picking a sequence of CSEs to 

## Next Steps

This is just one experiment.  The next step here is going to be using the accuracy of these models as the basis of comparison for evaluating the performance of neural networks on this same problem.  Depending on whether we find a neural network architecture that does as well as, or better than a random forest will determine our next steps.

If we are not able to find a neural network which better approximates this value, we can continue to try to refine this random forest approach.  The goal would be to build a classifier that "sees" the entire problem:  Selecting a sequence of CSEs, or at least has some concept of previous CSEs chosen and how many are viable to choose from.

With either forests or neural networks, we then need to compare it back to the base heuristic to see if we can either improve the heuristic, or consider incorporating these models into the JIT.  Libraries like `lime` help explain individual decisions in generally un-explainable models.  Later, we will find places where the hand-written heuristic does poorly but a `RandomForestClassifier` successfully determines what we should have done.  Then run that through `lime` to see if there were features that we should have considered to make the correct determination and update the hand-written heuristic with that approach.